<a href="https://colab.research.google.com/github/ChanceDurr/DS-Unit-1-Sprint-1-Dealing-With-Data/blob/master/module1-afirstlookatdata/LS_DS_111_A_First_Look_at_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Lambda School Data Science - A First Look at Data



## Lecture - let's explore Python DS libraries and examples!

The Python Data Science ecosystem is huge. You've seen some of the big pieces - pandas, scikit-learn, matplotlib. What parts do you want to see more of?

In [1]:
# TODO - we'll be doing this live, taking requests
# and reproducing what it is to look up and learn things

#This is a change to the notebook
2 + 2

4

## Assignment - now it's your turn

Pick at least one Python DS library, and using documentation/examples reproduce in this notebook something cool. It's OK if you don't fully understand it or get it 100% working, but do put in effort and look things up.

In [67]:
from bs4 import BeautifulSoup
import requests


def wordOnPage(url, word):
  page = requests.get(url).content #Use requests to get the content of the page
  soup = BeautifulSoup(page, 'html.parser') #input the content of the request into beautiful soup using python's html parser

  words = list((soup.get_text().split(" "))) #take all the text on the page and make each word a value in a list

  count = 0 #start the count var at 0

  for i in words: # for each time that word appears in the list, add 1 to the count
    if i.upper() == word.upper(): #using the .upper() function to get rid of capitalization discrepencies
      count += 1
  
  return f"The word \'{word}\' is on the page {count} times" # Return a string that tells you how many time a given word appears on a given page

website = 'https://www.crummy.com/software/BeautifulSoup/bs4/doc/'

wordOnPage(website, 'Beautiful')

"The word 'Beautiful' is on the page 108 times"

### Assignment questions

After you've worked on some code, answer the following questions in this text block:

1.  Describe in a paragraph of text what you did and why, as if you were writing an email to somebody interested but nontechnical.

2.  What was the most challenging part of what you did?

3.  What was the most interesting thing you learned?

4.  What area would you like to explore with more time?




    In the code above, what I did was create a '*Program*' that take a website page and a word that you give it and it tells you how many times that word appears on that page. I chose to use Beautiful Soup because I kept hearing about it and decided to look into what it was. From what I found, I also needed to use requests to get the HTML of the page to use in beautiful soup.
    
    The most challenging part of what I did was getting through the documentation to figure out what I needed to do to get the content of the page to appear in a str so that I could make a list of all the words on the page.
    
    The most interesting thing I learned probably was the requests part. It looks like you can do a lot with the GET and POST functions. I'm not exactly sure how they work, but I have a slight idea. 
    
    Based off of what I just worked on, I would like to explore more on website scraping to get data to create insights. 

  




## Stretch goals and resources

Following are *optional* things for you to take a look at. Focus on the above assignment first, and make sure to commit and push your changes to GitHub (and since this is the first assignment of the sprint, open a PR as well).

- [pandas documentation](https://pandas.pydata.org/pandas-docs/stable/)
- [scikit-learn documentation](http://scikit-learn.org/stable/documentation.html)
- [matplotlib documentation](https://matplotlib.org/contents.html)
- [Awesome Data Science](https://github.com/bulutyazilim/awesome-datascience) - a list of many types of DS resources

Stretch goals:

- Find and read blogs, walkthroughs, and other examples of people working through cool things with data science - and share with your classmates!
- Write a blog post (Medium is a popular place to publish) introducing yourself as somebody learning data science, and talking about what you've learned already and what you're excited to learn more about.